# Student Name: Lucy Tan

# Preface

Antiviral drugs are used for treating viral infections. The procedure often requires a sequence of treatments. Over time patients develop resistance to the drugs and need to keep switching them. Moreover, a drug of type 1 can make the patient (partially) resistant to the drug of type 2.

In this problem we consider drugs of type A, B, and C, all of which can potentially lower the viral load but the effect depends on their order. The 'Viral_load.xlsx' dataset provides results for 1,000 of patients, where rewards (based on the observed viral load level of patients) indicate the effectiveness of the preceding drugs used. For example, the first patient used drugs A, B, and C in this order and received the following rewards: 14.9062033, 13.67860579, and 12.76096513, respectively.

Because of the antiviral drug resistance, same drug cannot be used twice. The policy (behavior policy $b$) that was used for treatments represented in the 'Viral_load.xlsx' dataset is

1) if all drugs are available, pick A with probability 0.7, B with probability 0.2, and C with probability 0.1   
2) if all drugs, but A, are available, pick B with probability 0.65 and C with probability 0.35   
3) if all drugs, but B, are available, pick A with probability 0.9 and C with probability 0.1  
4) if all drugs, but C, are available, pick A with probability 0.8 and B with probability 0.2   
5) if only one drug is available, pick it with probability 1    



One can find that based on these observations, the optimal order of the treatment is B, A, and then C, indicating that drug A diminishes the effect of drug B, and C diminishes the effect of drug A (and possibly B).

The sequence of treatments can be represented as a Markov Decision Process (MDP). Clearly, the set of available actions at time t depends on the intire history. In order to model the treatment proceedure with the MDP, we introduce states as follows:

1) $S_{ABC}=${all drugs are available}   
2) $S_{BC}=${all drugs, except A, are available}   
3) $S_{AC}=${all drugs, except B, are available}       
4) $S_{AB}=${all drugs, except C, are available}   
5) $S_{A}=${only A is available}   
6) $S_{B}=${only B is available}   
7) $S_{C}=${only C is available}      

The optimal policy $\pi$, that corresponds to the sequence B, A, C, is then: (1) select B in state $S_{ABC}$; (2) select A in state $S_{AC}$; (3) select C in state $S_{C}$. The other actions do not need to be specified because the MDP will never reach them.
  

## Problem 1 (5 points)

In this problem, create a function that returns the importance-sampling ratio for each sequence of type "ABC" (use input format of choice), given target and behavior policies, $\pi$ and b.

In [129]:
import itertools

def admissible_states():
    # [(0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
    # where (0, 0, 1) means drug C is available
    states = list(itertools.product(*[[0, 1]] * 3))
    states.remove((0,0,0))
    return states

def action_index(action):
    location = {
        'A': 0,
        'B': 1,
        'C': 2
    }
    return location.get(action, None)

def is_admissible_action(action, state):
    # if state is (0, 0, 1), admissible actions are (0, 0, 1), meaning giving drug C is an admissible action
    return bool(state[action_index(action)])

def get_behavior_policy(state):
    if sum(state) == 1:
        policy_probs = state
    elif sum(state) == 2:
        probs_2 = {
            (1, 1, 0): (0.8, 0.2, None),
            (1, 0, 1): (0.9, None, 0.1),
            (0, 1, 1): (None, 0.65, 0.35)
        }
        policy_probs = probs_2.get(state)
    elif sum(state) == 3:
        policy_probs = (0.7, 0.2, 0.1)
    return policy_probs

def get_target_policy(state):
    probs = {
        (1, 1, 1): (0, 1, 0),
        (1, 1, 0): (0.5, 0.5, None),
        (1, 0, 1): (1, None, 0),
        (0, 1, 1): (None, 0.5, 0.5),
        (1, 0, 0): (1, None, None),
        (0, 1, 0): (None, 1, None),
        (0, 0, 1): (None, None, 1)
    }
    return probs.get(state, None)

def get_behavior_prob(action, state):
    return get_behavior_policy(state)[action_index(action)] if is_admissible_action(action, state) else None

def get_target_prob(action, state):
    return get_target_policy(state)[action_index(action)] if is_admissible_action(action, state) else None
    
def get_importance_sampling_ratio(sequence):
    state = [1, 1, 1]
    ratio = 1
    for action in sequence:
        target_prob = get_target_prob(action, tuple(state))
        behavior_prob = get_behavior_prob(action, tuple(state))
        if target_prob is not None and behavior_prob is not None:
            ratio = ratio * target_prob / behavior_prob
        else:
            ratio = None
            break
        state[action_index(action)] = 0
    return ratio
    
def sequences():
    return list(itertools.permutations(('A', 'B', 'C')))
    
importance_sampling_ratios = {}
for sequence in sequences():
    importance_sampling_ratios[sequence] = get_importance_sampling_ratio(sequence)
print(importance_sampling_ratios)

{('A', 'B', 'C'): 0.0, ('A', 'C', 'B'): 0.0, ('B', 'A', 'C'): 5.555555555555555, ('B', 'C', 'A'): 0.0, ('C', 'A', 'B'): 0.0, ('C', 'B', 'A'): 0.0}


## Problem 2 (10 points)

Using 'Viral_load.xlsx' dataset, estimate the state value function $v_\pi(s)$ for $s\in\{s_{ABC},s_{AC},s_{C}\}$ via Off-policy MC prediction (section 5.6 of "Reinforcement Learning" by Sutton and Barto). Please notice that generating an episode under policy $b$ would correspond to reading a row from 'Viral_load.xlsx'. Print the result for $\gamma=0.99$.

In [134]:
import collections
import csv

def off_policy_mc_prediction(episodes, gamma):
    V = collections.defaultdict(float)
    C = collections.defaultdict(float)
    for episode in episodes:
        state = [0, 0, 0]
        G = 0
        W = 1
        for time_step in episode[::-1]:
            action = time_step[0]
            reward = time_step[1]
            state[action_index(action)] = 1
            G = gamma*G+reward
            C[tuple(state)] += W
            V[tuple(state)] += (W/C[tuple(state)])*(G-V[tuple(state)])
            W = W*(get_target_prob(action, tuple(state))/get_behavior_prob(action, tuple(state)))  
    return V
        
def generate_episodes(csv_file):
    episodes = []
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in list(reader)[1:]:
            episode = [[row[1], float(row[2])],
                       [row[3], float(row[4])],
                       [row[5], float(row[6])]]
            episodes.append(episode)
    return episodes
    
csv_file = 'Viral_load.csv'
episodes = generate_episodes(csv_file)
states = [(1, 1, 1), (1, 0, 1), (0, 0, 1)]
gamma = 0.99
v = collections.defaultdict(float)
all_v = off_policy_mc_prediction(episodes, gamma)
for state in states:
    v[state] = all_v[state]
print(dict(v))

{(1, 1, 1): 40.76285747641467, (1, 0, 1): 27.705034462285298, (0, 0, 1): 14.025463818322676}


## Problem 3 (10 points)
Modify the algorithm you developed in Problem 2 in order to estimate $v_\pi(s)$ for $s\in\{s_{ABC},s_{AC},s_{C}\}$ via Off-policy one-step TD prediction. Print the result for $\gamma=0.99$ and $\alpha=0.15$.

In [135]:
def one_step_td_prediction(alpha, gamma, episodes):
    V = collections.defaultdict(float)
    for episode in episodes:
        state = [1, 1, 1]
        for time_step in episode:
            action = time_step[0]
            reward = time_step[1]
            next_state = state.copy()
            next_state[action_index(action)] = 0
            importance_sampling_ratio = get_target_prob(action, tuple(state))/get_behavior_prob(action, tuple(state))
            V[tuple(state)] += alpha*importance_sampling_ratio*(reward+gamma*V[tuple(next_state)]-V[tuple(state)])
            state = next_state
    return V

states = [(1, 1, 1), (1, 0, 1), (0, 0, 1)]
gamma = 0.99
alpha = 0.15
v = collections.defaultdict(float)
all_v = one_step_td_prediction(alpha, gamma, episodes)
for state in states:
    v[state] = all_v[state]
print(dict(v))

{(1, 1, 1): 41.01208121911871, (1, 0, 1): 27.7354244885284, (0, 0, 1): 13.897110376645768}
